# Custom Chatbot Project


The Arabic Poetry Dataset is an exceptional resource for exploring the richness and diversity of Arabic poetry, spanning over 70,000 poems by more than 750 poets from the 6th to the 20th century. It provides invaluable insights into the evolution of themes, styles, and poetic structures across different eras, enabling users to delve into the beauty of Arabic literary heritage. 

This dataset is particularly useful in scenarios where users seek accurate responses about poet names, their eras, and poetic meters—a challenge for many discovering Arabic poetry. For instance, it can be leveraged to build AI-powered tools for education, cultural preservation, or creative exploration, offering precise and contextually relevant information to poetry enthusiasts, researchers, and students.


## Data Wrangling


In [9]:
import pandas as pd
import openai
import tiktoken
from openai.embeddings_utils import get_embedding, distances_from_embeddings

openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "KEY"

In [2]:
data = pd.read_csv("Arabic_Poetry_Dataset.csv")
print(len(data))

data = data.loc[data["poem_tags"].apply(lambda x: str(x).count(","))==3].copy()

data["poem_meter"] = data["poem_tags"].apply(lambda x: x.split(",")[2])

data.head()

75022


,poet_name,poet_era,poem_tags,poem_title,poem_text,poem_count,poem_meter
0,لقيط بن يعمر الإيادي,العصر الجاهلي,"قصائد هجاء, عموديه, بحر الوافر, قافية الدال (د)",سلام في الصحيفة من لقيط,سَلامٌ في الصَحيفَةِ مِن لَقيطٍ\nإِلى مَن بِال...,عدد الابيات : 4,بحر الوافر
1,لقيط بن يعمر الإيادي,العصر الجاهلي,"قصائد حزينه, عموديه, بحر البسيط, قافية الألف ...",يا دار عمرة من محتلها الجرعا,يا دارَ عَمرَةَ مِن مُحتَلِّها الجَرَعا\nهاجَت...,عدد الابيات : 60,بحر البسيط
2,لقيط بن يعمر الإيادي,العصر الجاهلي,"قصائد قصيره, عموديه, بحر الرجز, قافية الألف (ا)",وخاننا خوان في ارتباعنا,وَخانَنا خَوّانٌ في اِرتِباعِنا\nفَاِنفَدَّ لِ...,عدد الابيات : 1,بحر الرجز
3,زبان بن سيار الفزاري,العصر الجاهلي,"قصائد قصيره, عموديه, بحر الطويل, قافية الألف ...",تنح إليكم يا ابن كوز فإننا,تَنَحَّ إِلَيكُم يا اِبنَ كوزٍ فَإِنَّنا\nوَإِ...,عدد الابيات : 1,بحر الطويل
4,زبان بن سيار الفزاري,العصر الجاهلي,"قصائد قصيره, عموديه, بحر الطويل, قافية الباء ...",تطارحه الأنساب حتى رددنه,تُطارِحُهُ الأَنسابُ حَتّى رَدَدنَهُ\nإِلى نَس...,عدد الابيات : 1,بحر الطويل


In [3]:
# to reduce the number of rows
txt = 'المجد'
data = data.loc[ data["poem_title"].str.contains(txt)].reset_index(drop=True)

In [6]:
df = pd.DataFrame(columns = ["text"])
df["text"] = data.apply(lambda x : "اسم الشاعر: " + x["poet_name"] + " , " + "العصر: " +  x["poet_era"] + " , " + "البحر: " +  x["poem_meter"] + " , " + "العنوان: " +  x["poem_title"] , axis=1) # + " , " + "الابيات: " +  x["poem_text"]
print(len(df))
df.head()

170


,text
0,"اسم الشاعر: زبان بن سيار الفزاري , العصر: العص..."
1,"اسم الشاعر: عبد الله بن الزبعرى , العصر: المخض..."
2,"اسم الشاعر: حسان بن ثابت , العصر: المخضرمون , ..."
3,"اسم الشاعر: حسان بن ثابت , العصر: المخضرمون , ..."
4,"اسم الشاعر: علي بن أبي طالب , العصر: المخضرمون..."


In [7]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []

for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df

,text,embeddings
0,"اسم الشاعر: زبان بن سيار الفزاري , العصر: العص...","[-0.006997411604970694, 0.007916332222521305, ..."
1,"اسم الشاعر: عبد الله بن الزبعرى , العصر: المخض...","[-0.005346281919628382, -0.0020473382901400328..."
2,"اسم الشاعر: حسان بن ثابت , العصر: المخضرمون , ...","[-0.0035146602895110846, 0.0037174930330365896..."
3,"اسم الشاعر: حسان بن ثابت , العصر: المخضرمون , ...","[-0.007776329759508371, 7.898877083789557e-05,..."
4,"اسم الشاعر: علي بن أبي طالب , العصر: المخضرمون...","[0.00027684596716426313, -0.003262293059378862..."
...,...,...
165,"اسم الشاعر: أبو الحسن السلامي , العصر: العصر ا...","[-0.014789328910410404, -0.005357156042009592,..."
166,"اسم الشاعر: ابن نباتة السعدي , العصر: العصر ال...","[-0.014750099740922451, 0.0031649877782911062,..."
167,"اسم الشاعر: ابو الحسن السلامي , العصر: العصر ا...","[-0.013343694619834423, -0.004649070557206869,..."
168,"اسم الشاعر: أبو بكر الخالدي , العصر: العصر الع...","[-0.009930785745382309, 0.006275240797549486, ..."


## Custom Query Completion

In [19]:
def get_rows_sorted_by_relevance(question):

    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)

    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )

    df_copy.sort_values("distances", ascending=True, inplace=True)

    return df_copy["text"].values

In [46]:
tokenizer = tiktoken.get_encoding("cl100k_base")

prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know". Then Paraphrase the answer in formal tone and grammatically correct. 

Context: 

{}

---

Question: {}
Answer:"""

def create_prompt(question, max_token_count):
    current_token_count= len(tokenizer.encode(prompt_template)) + len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question):
        
        current_token_count += len(tokenizer.encode(text))
        
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break
            
    return prompt_template.format("\n\n###\n\n".join(context), question)
    

In [12]:
def answer_question(question, model, max_tokens, max_prompt_tokens=1800):

    prompt = create_prompt(question, max_prompt_tokens)
    
    try:
        response = openai.Completion.create(
            prompt=prompt,
            model=model,
            max_tokens=max_tokens
        )
        return response["choices"][0]["text"].strip()
    
    except Exception as e:
        print(e)
        return ""

## Custom Performance Demonstration

In [13]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"
MAX_TOKENS = 200

### Question 1

In [25]:
QUESTION_1 = """
ما هو بحر قصيدة ألا في سبيل المجد ما أنا فاعل... ؟ ومن الشاعر ؟ وفي أي عصر كتبت ؟
"""

In [26]:
response = openai.Completion.create(
    prompt=QUESTION_1,
    model=COMPLETION_MODEL_NAME,
    max_tokens=MAX_TOKENS)

print(response["choices"][0]["text"].strip())

بحر قصيدة "ألا في سبيل المجد ما أنا فاعل" هو البحر السريع (albaḥr al-sarīʿ) المنزلق (al-manzilq) وينتمي إلى الفرع التجميعي للأبحاث الصغيرة.

الشاعر الذي ألف هذه القصيدة هو أبو الطيب أحمد بن حزام الإسكندري، المعروف باسم ابن الفارض. ولد في القرن الثالث عشر الميلادي في مدينة إسكندرية في مصر وتوفي ح


This answer is incorrect!

In [47]:
answer_question(QUESTION_1, model=COMPLETION_MODEL_NAME, max_tokens=MAX_TOKENS)

'بحر قصيدة "ألا في سبيل المجد ما أنا فاعل" هو بحر الطويل وكتبها الشاعر أبو العلاء المعري في العصر العباسي.'

This is the correct answer!

### Question 2

In [49]:
QUESTION_2 = """
قصيدة لا يمتطي المجد من لم يركب الخطرا، من الشاعر؟ وفي أي عصر كتبت ؟ وعلى أي بحر كتبت هذه القصيدة ؟
"""

In [52]:
MAX_TOKENS = 400

In [50]:
response = openai.Completion.create(
    prompt=QUESTION_2,
    model=COMPLETION_MODEL_NAME,
    max_tokens=MAX_TOKENS)

print(response["choices"][0]["text"].strip())

لا يمتطي المجد من لم يركب الخطرا،
ما أخذ النيل من الماء إذ لا يسرا
للأصل تقبلوا فأنتم من أنشأنا
فيالصدر جرت صلاة الخطى بسرا
إذا عرفت نفسك لم تحيد يوما
فاصلط الأمر هوى المسير واخترى
 
هذه القصيدة من تأليف الشاعر العربي الجاهلي العالمة الغابري، وتعتبر من القصائد الشعرية العربية القدي


This answer is incorrect ! 

لا يَمتَطي المَجدَ مَن لَم يَركَبِ الخَطَرا ... وَلا يَنالُ العُلا مَن قَدَّمَ الحَذَرا

وَمَن أَرادَ العُلا عَفواً بِلا تَعَبٍ ... قَضى وَلَم يَقضِ مِن إِدراكِها وَطَرا

In [53]:
answer_question(QUESTION_2, model=COMPLETION_MODEL_NAME, max_tokens=MAX_TOKENS)

'لا تعرف قصيدة الشاعر الحلي بإسمي صفي الدين الحلي و هو من العصر المملوكي، و كتب على بحر البسيط و تحمل العنوان لا يمتطي المجد من لم يركب الخطرا.'

This is the correct answer !